In [17]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import math
import requests
import json
import pyodbc
import streamlit as st
import math

# Define your SQL Server connection parameters


def data_generation_synapse(sql_command):
    server = 'LAPTOP-2MROT4G8\SQLEXPRESS'
    database = 'master'
    connection_string = f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
    
    # Create an SQLAlchemy engine
    engine = create_engine(connection_string)

    # Use the engine to execute the SQL command and read the result into a DataFrame
    df_orders = pd.read_sql(sql_command, con=engine)
    return df_orders



sql_command = """
                select * from ratecard
              """

ratecard = data_generation_synapse(sql_command)




In [2]:
import requests
import json

def get_zone_chart(zip_code, shipping_date):
    # Construct the URL with the given ZIP code and shipping date
    url = f"https://postcalc.usps.com/DomesticZoneChart/GetZoneChart?zipCode3Digit={zip_code}&shippingDate={shipping_date}"

    # Optional headers (these can be customized if necessary)
    headers = {
        'authority': 'postcalc.usps.com',
        'method': 'GET',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }

    # Send the GET request to the USPS API
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # Handle HTTP errors
        return None
    except Exception as err:
        print(f"Other error occurred: {err}")  # Handle other errors
        return None
    
    # Parse the JSON data if the request was successful
    try:
        data = response.json()
        return data
    except json.JSONDecodeError:
        print("Failed to parse JSON response.")
        return None

# Example usage
zip_code = "700"  # Example ZIP code
shipping_date = "8/12/2024"  # Example shipping date in MM/DD/YYYY format

response_data = get_zone_chart(zip_code, shipping_date)

if response_data:  # Corrected this line
    print("Response Data:", response_data)
    print(type(response_data))
    print(response_data)
else:
    print("No valid data received.")


Response Data: {'ZIPCodeError': '', 'ShippingDateError': '', 'PageError': '', 'EffectiveDate': 'August 1, 2024', 'Column0': [{'ZipCodes': '005', 'Zone': '6', 'MailService': ''}, {'ZipCodes': '006---009', 'Zone': '7', 'MailService': ''}, {'ZipCodes': '010---034', 'Zone': '6', 'MailService': ''}, {'ZipCodes': '035', 'Zone': '7', 'MailService': ''}, {'ZipCodes': '036---037', 'Zone': '6', 'MailService': ''}, {'ZipCodes': '038---049', 'Zone': '7', 'MailService': ''}, {'ZipCodes': '050---057', 'Zone': '6', 'MailService': ''}, {'ZipCodes': '058---059', 'Zone': '7', 'MailService': ''}, {'ZipCodes': '060---089', 'Zone': '6', 'MailService': ''}, {'ZipCodes': '090---099', 'Zone': '6+', 'MailService': ''}, {'ZipCodes': '100---149', 'Zone': '6', 'MailService': ''}, {'ZipCodes': '150---157', 'Zone': '5', 'MailService': ''}, {'ZipCodes': '158', 'Zone': '6', 'MailService': ''}, {'ZipCodes': '159---162', 'Zone': '5', 'MailService': ''}, {'ZipCodes': '163---165', 'Zone': '6', 'MailService': ''}, {'ZipCo

In [4]:


def process_response_data(response_data):
    # Extract the relevant parts of the JSON data
    columns_data0 = response_data['Column0']
    columns_data1 = response_data['Column1']
    columns_data2 = response_data['Column2']
    # Zip5Digit = response_data['Zip5Digit']  # Uncomment if needed

    # Convert the extracted data to DataFrames
    columns_data0_df = pd.DataFrame(columns_data0)
    columns_data1_df = pd.DataFrame(columns_data1)
    columns_data2_df = pd.DataFrame(columns_data2)
    # Zip5Digit_df = pd.DataFrame(Zip5Digit)  # Uncomment if needed

    # Concatenate the DataFrames along the rows
    combined_df = pd.concat([columns_data0_df, columns_data1_df, columns_data2_df], axis=0)

    # Return a copy of the combined DataFrame
    return combined_df.copy()

# Example usage:
response_data = get_zone_chart(zip_code, shipping_date)
df = process_response_data(response_data)


In [5]:
# Function to expand ZIP code ranges
def expand_zipcodes(zip_code):
    if '---' in zip_code:
        start, end = zip_code.split('---')
        return [str(i).zfill(3) for i in range(int(start), int(end) + 1)]
    else:
        return [zip_code]

# Expand the ZIP codes based on the existing DataFrame
expanded_zipcodes = []

for i, row in df.iterrows():
    zip_codes = expand_zipcodes(row['ZipCodes'])
    for zip_code in zip_codes:
        expanded_zipcodes.append({'ZipCode': zip_code, 'Zone': row['Zone']})

# Create a new DataFrame with expanded ZIP codes
expanded_df = pd.DataFrame(expanded_zipcodes)


In [18]:
import math

def calculate_billable_weight(l, b, h, d):
    """
    Calculate the billable weight based on dimensions and actual weight.

    Parameters:
    l (float): Length in inches
    b (float): Width in inches
    h (float): Height in inches
    d (float): Actual weight in ounces

    Returns:
    int: Billable weight in ounces
    """
    # Ensure all dimensions and weight are in numeric form
    l = float(l)
    b = float(b)
    h = float(h)
    d = float(d)

    # Calculate dimensional weight in ounces
    Dimweight = ((l * b * h) / 166) * 16

    # Calculate billable weight
    Billableweight = max(math.ceil(Dimweight), math.ceil(d))

    return Billableweight

# Example usage
l = 20  # length in inches
b = 15  # width in inches
h = 10  # height in inches
d = 25  # actual weight in ounces

billable_weight = calculate_billable_weight(l, b, h, d)
print(billable_weight)


290
